# Majority voting

In [1]:
import os
import torch
from vllm import LLM, SamplingParams
from datasets import load_dataset
from collections import defaultdict
import torch.multiprocessing as mp
import random, numpy as np
import time
from utils import *

/root/miniconda/envs/sal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-03 17:11:47,606	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/miniconda/envs/sal/lib/python3.11/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


* Setting Configurations 

In [2]:
config = {
    "model_path": "meta-llama/Llama-3.1-8B-Instruct",
    "dataset_name": "HuggingFaceH4/MATH-500",
    "dataset_split": "test",
    "prompt": """Solve the following math problem efficiently and clearly:

- For simple problems (2 steps or fewer):
Provide a concise solution with minimal explanation.

- For complex problems (3 steps or more):
Use this step-by-step format:

## Step 1: [Concise description]
[Brief explanation and calculations]

## Step 2: [Concise description]
[Brief explanation and calculations]

...

Therefore, the final answer is: $\\boxed{answer}$. I hope it is correct.

Where [answer] is just the final number or expression that solves the problem."""
}

In [3]:
# Fix seed 
def set_seed(seed):
    mp.set_sharing_strategy('file_system')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [4]:
# Start llm engine 
llm = LLM(model=config["model_path"])

WARNING 08-03 08:37:38 arg_utils.py:953] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-03 08:37:38 config.py:1005] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-03 08:37:38 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, qu

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  3.29it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.47it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.25it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.23it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.32it/s]


INFO 08-03 08:37:45 model_runner.py:1071] Loading model weights took 14.9888 GB


INFO 08-03 08:37:46 gpu_executor.py:122] # GPU blocks: 28041, # CPU blocks: 2048
INFO 08-03 08:37:46 gpu_executor.py:126] Maximum concurrency for 131072 tokens per request: 3.42x
INFO 08-03 08:37:47 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-03 08:37:47 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-03 08:37:55 model_runner.py:1530] Graph capturing finished in 8 secs.


# Test trials for majority voting

In [15]:
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()  # Jupyter 환경에서 필수

from typing import List

# Config 설정
api_url = "http://localhost:8000/v1/chat/completions"
model_name = "meta-llama/Llama-3.1-8B-Instruct"
context_len = 1500

# 개별 요청 함수
async def send_prompt(session: aiohttp.ClientSession, prompt: str, idx: int, context_len: int) -> str:
    payload = {
        "model": model_name,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.8,
        "top_p": 1.0,
        "min_tokens": context_len-1,
        "max_tokens": context_len
    }
    try:
        async with session.post(api_url, json=payload) as resp:
            data = await resp.json()
            return data["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"[{idx}] request failed: {e}")
        return ""

# 병렬 요청 함수
async def generate_n_responses(prompt: str, n: int) -> List[str]:
    async with aiohttp.ClientSession() as session:
        tasks = [send_prompt(session, prompt, i, context_len) for i in range(n)]
        return await asyncio.gather(*tasks)



In [16]:
# 사용 예시
prompt = "What is the capital of France?"

# 실행
responses = await generate_n_responses(prompt, n=4)
for i, r in enumerate(responses):
    print(f"[{i}] {r}")

[0] The capital of France is Paris. It is the country's largest city and is known for its famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. It is also a major cultural and economic hub, and is home to many of France's government institutions, universities, and cultural attractions. Paris has been the capital of France since 987. It is one of the most visited cities in the world and is known for its romantic atmosphere, fashion, art, and cuisine. It has also been the hub of many historical events and revolutions, and is home to many significant museums, galleries, and libraries. 

If you are looking for more information, I would be happy to help. Let me know what you would like to know. 

Is there something specific you would like to know? Would you like to know more about Paris in general, or perhaps one of the cities attractions? Let me know if you would like more information. 

I can also help you with more questions if you would like. Do you hav

In [ ]:
from transformers import AutoTokenizer

# LLaMA 3.1 tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")



Token count: 1501


In [22]:
# 응답 예시
text = responses[3]

# 토큰 수 계산
token_count = len(tokenizer.encode(text))
print(f"Token count: {token_count}")

Token count: 1501


In [18]:
responses[0]

"The capital of France is Paris. It is the country's largest city and is known for its famous landmarks such as the Eiffel Tower, Notre Dame Cathedral, and the Louvre Museum. It is also a major cultural and economic hub, and is home to many of France's government institutions, universities, and cultural attractions. Paris has been the capital of France since 987. It is one of the most visited cities in the world and is known for its romantic atmosphere, fashion, art, and cuisine. It has also been the hub of many historical events and revolutions, and is home to many significant museums, galleries, and libraries. \n\nIf you are looking for more information, I would be happy to help. Let me know what you would like to know. \n\nIs there something specific you would like to know? Would you like to know more about Paris in general, or perhaps one of the cities attractions? Let me know if you would like more information. \n\nI can also help you with more questions if you would like. Do you 

In [5]:
from typing import List, Dict, Tuple
from collections import defaultdict

# Majority voting with normalization
def find_majority_answer(completions: List[str]) -> Tuple[str, Dict[str, int]]:
    canonical_groups = defaultdict(int)
    canonical_to_original = {}

    for output in completions:
        # Step 1: Try extracting from boxed expression
        boxed = last_boxed_only_string(output)
        if boxed:
            try:
                answer = remove_boxed(boxed)
            except Exception:
                continue  # skip this sample if boxed parsing fails
        else:
            continue  # skip if no boxed answer found

        # Step 2: Normalize and vote
        canonical = strip_string(answer)
        canonical_groups[canonical] += 1

        if canonical not in canonical_to_original:
            canonical_to_original[canonical] = answer

    # Step 3: Determine majority
    if not canonical_groups:
        return "", {}  # No valid votes

    max_count = max(canonical_groups.values())
    for canonical, count in canonical_groups.items():
        if count == max_count:
            return canonical_to_original[canonical], dict(canonical_groups)

# Inference wrapper with prompt formatting
def majority_vote(batch: Dict, config: Dict, sampling_params, llm) -> Dict:
    prompt = batch["problem"]
    full_prompt = f"{config['prompt'].strip()}\n\n### Problem:\n{prompt.strip()}"

    # generate가 List[RequestOutput] 반환
    request_outputs = llm.generate(full_prompt, sampling_params=sampling_params)

    # 각 RequestOutput 객체에서 outputs 리스트를 펼쳐서 CompletionOutput.text 수집
    completions = []
    for request_output in request_outputs:
        completions.extend([output.text for output in request_output.outputs])

    pred, votes = find_majority_answer(completions)

    return {
        "completions": completions,
        "pred": pred,
        "votes": votes,
    }

# Prediction vs GT comparison
def evaluate_single(predicted: str, gt_solution: str) -> bool:
    pred_norm = strip_string(predicted)
    try:
        gt_extracted = remove_boxed(last_boxed_only_string(gt_solution)) or gt_solution
        gt_norm = strip_string(gt_extracted)
    except Exception:
        gt_norm = strip_string(gt_solution)
    return pred_norm == gt_norm

In [6]:
from datasets import Dataset, load_dataset

dataset = load_dataset(config['dataset_name'], split=config['dataset_split'])
dataset


Dataset({
    features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
    num_rows: 500
})

# Power monitoring & save module

In [7]:
import os
import json
import pandas as pd
from pathlib import Path
from typing import List


# Base directory to store all results
base_dir = Path("results")
base_dir.mkdir(exist_ok=True)

# Helper function to save result and CSV placeholder
def save_results_and_csv(problem_id: int, n: int, result: dict, is_correct: bool):
    # Define directory
    folder_name = f"problem_{problem_id}_batch_{n}"
    folder_path = base_dir / folder_name
    folder_path.mkdir(exist_ok=True)

    # Save majority_vote result + correctness
    result_to_save = {
        "pred": result["pred"],
        "votes": result["votes"],
        "completions": result["completions"],
        "is_correct": is_correct
    }
    with open(folder_path / "result.json", "w") as f:
        json.dump(result_to_save, f, indent=2)

    # Create placeholder CSV for power monitoring
    csv_path = folder_path / "power_log.csv"
    if not csv_path.exists():
        df = pd.DataFrame(columns=["timestamp", "power_W"])
        df.to_csv(csv_path, index=False)

# Show a sample path
base_dir.absolute()

PosixPath('/root/search-and-learn/scripts/results')

# Main experiment

In [ ]:
from monitor_power import monitor_npu_power_usage, monitor_gpu_power_usage, calculate_avg_power_usage
from multiprocessing import Process, Queue
from datetime import datetime

n_values = [1, 2, 4, 8, 16, 32, 48, 64, 80, 96, 112, 128]

# 전체 루프
for n in n_values:
    for idx in range(500):
    
        # 1. 폴더 생성
        run_dir = f"results/batch_{n}/problem_{idx}"
        os.makedirs(run_dir, exist_ok=True)
        power_monitor_path = os.path.join(run_dir, "device_status.csv")

        # 2. Power monitor start
        stop_queue = Queue()
        monitor_process = Process(
            target=monitor_gpu_power_usage,  # or monitor_npu_power_usage
            args=(power_monitor_path, stop_queue)
        )
        monitor_process.start()

        # 3. Run inference
        sampling_params = SamplingParams(
            n=n,
            temperature=0.8,
            top_p=1.0,
            max_tokens=2048,
        )

        try:
            result = majority_vote(dataset[idx], config, sampling_params, llm)
            is_correct = is_equiv(result["pred"], dataset[idx]['answer'], verbose=False)
        except Exception as e:
            result = {"error": str(e)}
            is_correct = False

        # 4. Stop power monitor
        stop_queue.put("stop")
        monitor_process.join()

        # 5. 평균 전력 사용량 계산
        avg_power = calculate_avg_power_usage(power_monitor_path)

        # 6. 결과 저장
        output_path = os.path.join(run_dir, "result.json")
        with open(output_path, "w") as f:
            json.dump({
                "index": idx,
                "batch_size": n,
                "prediction": result.get("pred"),
                "votes": result.get("votes"),
                "is_correct": is_correct,
                "avg_power_usage": avg_power,
                "error": result.get("error", None)
            }, f, indent=2)

        print(f"✅ Finished problem_{idx}, batch_{n}, correct={is_correct}")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.48s/it, est. speed input: 10.73 toks/s, output: 132.35 toks/s]

✅ Finished problem_0, batch_1, correct=True



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.27s/it, est. speed input: 10.87 toks/s, output: 268.24 toks/s]


✅ Finished problem_0, batch_2, correct=False


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processed prompts: 100%|██████████| 1/1 [00:25<00:00, 25.56s/it, est. speed input: 6.49 toks/s, output: 4660.41 toks/s]


✅ Finished problem_0, batch_80, correct=True


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Monitoring stopped.


KeyboardInterrupt: 